# 01 Time Series Basics

Resampling, lags, rolling windows, and leakage intuition.


## Table of Contents
- Toy series
- Resampling
- Lag and rolling features
- Leakage demo


## Why This Notebook Matters
Foundations notebooks build the intuition that prevents the most common mistakes in economic ML:
- leaking future information,
- evaluating with the wrong split strategy,
- over-interpreting coefficients.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Using random splits on time series.
- Assuming correlation implies causation.

## Matching Guide
- `docs/guides/00_foundations/01_time_series_basics.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/00_foundations/01_time_series_basics.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Concept
You will build intuition for resampling, lags, rolling windows, and leakage.


## Your Turn: Build a Toy Time Series


In [ ]:
import numpy as np
import pandas as pd

# TODO: Create a daily toy series with a trend + noise
# Hint: pd.date_range(..., freq='D')
...


## Your Turn: Resample


In [ ]:
# TODO: Resample to month-end and compute monthly mean and last
...


## Your Turn: Lags + Rolling


In [ ]:
# TODO: Create lag-1 and lag-7 features
# TODO: Create a 14-day rolling mean
...


## Leakage Demo


In [ ]:
# TODO: Create a target that is tomorrow's value
# TODO: Compare a random split vs time split
# Write down what changes and why
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run a quick sanity check on any DataFrame/Series you created in this notebook.
# Example (adjust variable names):
# assert df.index.is_monotonic_increasing
# assert df.isna().sum().sum() == 0
#
# TODO: Write 2-3 sentences:
# - What would leakage look like in YOUR code?
# - How would you detect it?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Toy series</summary>

```python
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
idx = pd.date_range('2020-01-01', periods=365, freq='D')
y = pd.Series(0.01*np.arange(len(idx)) + rng.normal(0, 1, len(idx)), index=idx, name='y')
y.head()
```

</details>

<details><summary>Solution: Resampling</summary>

```python
monthly_mean = y.resample('ME').mean()
monthly_last = y.resample('ME').last()
monthly_mean.head(), monthly_last.head()
```

</details>

<details><summary>Solution: Lag and rolling features</summary>

```python
df = y.to_frame()
df['y_lag1'] = df['y'].shift(1)
df['y_lag7'] = df['y'].shift(7)
df['y_roll14_mean'] = df['y'].rolling(14).mean()
df = df.dropna()
df.head()
```

</details>

<details><summary>Solution: Leakage demo</summary>

```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

tmp = df.copy()
tmp['target_tomorrow'] = tmp['y'].shift(-1)
tmp = tmp.dropna()

X = tmp[['y_lag1', 'y_lag7', 'y_roll14_mean']]
y_t = tmp['target_tomorrow']

# Random split (not recommended for time series)
X_tr, X_te, y_tr, y_te = train_test_split(X, y_t, test_size=0.2, shuffle=True, random_state=0)
m = LinearRegression().fit(X_tr, y_tr)
rmse_rand = mean_squared_error(y_te, m.predict(X_te), squared=False)

# Time split (recommended)
split = int(len(tmp) * 0.8)
m2 = LinearRegression().fit(X.iloc[:split], y_t.iloc[:split])
rmse_time = mean_squared_error(y_t.iloc[split:], m2.predict(X.iloc[split:]), squared=False)

rmse_rand, rmse_time
```

</details>

